In [ ]:
import h5py
import numpy as np

##load dataset
train_dataset_ptr = h5py.File('datasets/train_signs.h5', "r")
test_dataset_ptr = h5py.File('datasets/test_signs.h5', "r")

#train_set set x and y
train_set_x_orig = np.array(train_dataset_ptr["train_set_x"][:])
train_set_y_orig = np.array(train_dataset_ptr["train_set_y"][:])

#testset also
test_set_x_orig = np.array(test_dataset_ptr["test_set_x"][:])
test_set_y_orig = np.array(test_dataset_ptr["test_set_y"][:])

# classes
classes = np.array(test_dataset_ptr["list_classes"][:])


### EXERCISE
- 패딩 넣기
패딩공간에 0을 넣는걸 ZERO_PADDING이라고 합니다
np.pad함수를 사용함으로써 원하는 숫자를 원하는 곳에 넣을 수 있습니다.

- np.pad 예시
X가 이미지이고 200,200,3(RGB) 의 SHAPE를 가진다고 해보죠.
X에 2만큼의 0 패딩을 넣고 싶다면
np.pad(원본이미지, 어느 차원에 얼마만큼 넣을지, 모드, 패딩에 들어갈 숫자) 로 사용하면 됩니다.

padded_img = np.pad(X, ((2,2),(2,2),(0,0)), mode= 'constant', constant_values=(0,0)) 과 같은 식로 사용합니다

In [20]:
#exercise1 zero_pad
def zero_pad(X, pad):
    '''
    input : x, pad (패딩을 얼만큼 적용할 것인지)
    이미지x는 m, n_h, n_W, n_C로 구성되어 있습니다
    m : 배치사이즈 
    n_h : 이미지의 높이 number of height
    n_w : 이미지의 너비 number of weidth
    n_c : 이미지 채널 수 number of channel
    '''
    x_pad = np.pad(X,(
        (0,0), (pad,pad),(pad,pad), (0,0)),
        mode = 'constant',
        constant_values = (0,0),
    )
    return x_pad

In [39]:
def conv_single_step(a_slice_prev, W, b):
    '''
    컨볼루전의 과정을 구현
    A가 슬라이스되어서 들어올거임.
    가중치'화'된 필터를 element_wise연산으로 곱한다음 그 결과를 더해서 도출
    '''
    s = W * a_slice_prev
    Z = np.sum(s) + b.item()
    return Z
    

In [ ]:
def conv_forward(A_prev, W, b, hparameters):
    '''
    A_prev: 말 그대로 A[l-1] 
    W,b: 가중치'화'된 필터블럭
    hparameters: 스트라이드와 패드가 있는 dictioar
    Output: Z ,cache 
    Z는 활성함수가 적용된 상태

    컨볼루션 계산 과정을 직접 구현할 것임. 
    슬라이싱을 이용하면 편하겠죠?
    '''
    #A_prev의 shape을 알아야겠지요
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape

    #W의 shape도 알아봅시다
    #단일 w는 (f, f, n_C_prev)로 구성되어 있고, 
    #W = (w1,w2,..필터의 갯수) 만큼 되어있기 때문에
    #W.shape는 (f, f, n_C_prev, n_C)입니다
    (f, f, n_C_prev, n_C) = W.shape

    #하이퍼파라미터 dict에서 int로 저장된 stride와 pad를 불러옵니다.
    stride = hparameters["stride"]
    pad = hparameters["pad"]

    #conv 아웃풋의 차원 수를 계산합니다
    n_H = int((n_H_prev +2 * pad - f)/stride) + 1
    n_W = int((n_W_prev +2 * pad - f)/stride) + 1

    Z = np.zeros((m,n_H,n_W,n_C))

    A_prev_pad = zero_pad(A_prev, pad)
    


